In [1]:
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from matplotlib import pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

import bo_methods_lib
from bo_methods_lib.bo_functions_generic import gen_theta_set

from bo_methods_lib.CS2_bo_plotters import plot_obj_abs_min, value_plotter, many_value_plotter, plot_obj, plot_Theta, plot_Theta_min, path_name, plot_org_train, save_fig, plot_EI_abs_max

from bo_methods_lib.CS2_bo_functions_multi_dim import set_ep

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html

def csv_to_array(csv_path):
    """
    Turn a csv file into a numpy array so that it can be used with bo_plotters
    
    Parameters:
    -----------
        csv_path: str, path of csv you want to turn into an array
    Returns:
    --------
        csv_array: np.ndarray, array on values in the CSV file
    """
    
    csv_pd = pd.read_csv(csv_path, index_col = False)
    csv_pd.drop(columns=csv_pd.columns[0], 
        axis=1, 
        inplace=True)
    csv_array = csv_pd.to_numpy()
    
    return csv_array

In [11]:
# DateTime = "2022/10/21/13-05" #Date and Time - Dispayed in job file
# DateTime = "2022/10/21/13-18" #Date and Time - Dispayed in job file
DateTime = "2022/12/02/19-19" #Date and Time - Dispayed in job file
DateTime = "../2023/05/10/09-16" #Date and Time - Dispayed in job file
DateTime = "../2023/05/09/11-58" #Date and Time - Dispayed in job file
# DateTime = None

emulator = True
obj = "obj"
sparse_grid = True
normalize = False
sep_fact = 1.0

exp_d = 1
n = 5

    
skip_param_types = 0
param_dict = {0 : '\\theta_1', 1 : '\\theta_2'}
bounds_x = np.array([[-2], [2]])
bounds_p = np.array([[-2, -2],
                         [ 2,  2]])
indecies = [list(param_dict.keys())[list(param_dict.values()).index('\\theta_1')],
            list(param_dict.keys())[list(param_dict.values()).index('\\theta_2')]]

Theta_True = np.array([1,-1])

bounds = np.array([[-2, -2],
                   [ 2,  2]])

Bound_Cut = False
denseX = False

param_names_list = [param_dict[0], param_dict[1]] #Fill in depending on problem
mesh_combo = str(param_names_list[0]) + "-" + str(param_names_list[1])

# run = 1
# Bo_iter = 1

run = 7
Bo_iter = 1

set_lengthscale = None
save_CSV = False

t = 20
if emulator == True:
    t = n*t
    
if Bound_Cut == True:
    cut_bounds = '_cut_bounds'
else:
    cut_bounds = ""

if denseX == True:
    dense = "_dense"
else:
    dense = ""

#Pull Experimental data from CSV
exp_data_doc = '../Input_CSVs/Exp_Data/d='+str(exp_d)+'/n='+str(n)+cut_bounds+dense+'.csv'
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1:exp_d+1]
Yexp = exp_data[:,-1]

#Define GP Testing space
p=20
d = len(Theta_True)
LHS = True
# Need to redefine to save theta_set_org 
theta_set_org = gen_theta_set(LHS = LHS, n_points = p, dimensions = d, bounds = bounds_p, seed =1)
Theta1_lin = np.linspace(np.min(theta_set_org[:,indecies[0]]),np.max(theta_set_org[:,indecies[0]]), p)
Theta2_lin = np.linspace(np.min(theta_set_org[:,indecies[1]]),np.max(theta_set_org[:,indecies[1]]), p)
theta_mesh = np.array(np.meshgrid(Theta1_lin, Theta2_lin)) 

# theta_mesh = gen_theta_set(LHS = False, n_points = p, dimensions = d, bounds = bounds)
# theta_mesh = theta_mesh.T.reshape((2, p, p))
# # print(theta_mesh.shape)
    
# ep = set_ep(emulator, obj, sparse_grid)
# print(ep)
ep = 1

tot_iter = bo_iters= 100 #Total Number of iters - Dispayed in job file
tot_runs = 15 #Total Number of runs - Dispayed in job file

# tot_iter = bo_iters= 100 #Total Number of iters - Dispayed in job file
# tot_runs = 15 #Total Number of runs - Dispayed in job file

is_figure = False
save_figure = False

run -= 1 # run should be number you want -1
Bo_iter -= 1 #Iter you want -1

In [12]:
# save_figure = False
save_figure = True
# titles = ['E(I(\\theta))','log(e(\\theta))','\sigma^2','\sigma']  
# titles_save = ["EI","ln(SSE)","Var","StDev"]


for Bo_iter in range(tot_iter):
    fxn = "value_plotter" #Information saved using this function so we must pull it using this function name

    title_save = ["EI", "ln(SSE)", "StDev", "Var"]
    title = ['E(I(\\theta))', 'log(e(\\theta))', '\sigma', '\sigma^2']
    levels = [200,100, 100, 100]

    title_save = ["EI", "ln(SSE)"]
    title = ['E(I(\\theta))', 'log(e(\\theta))']
    levels = [200, 100]

    z = np.zeros((len(title), p, p))

    for i in range(len(title)):
        path_title = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo, Bo_iter, title_save[i], run, tot_iter=tot_iter, 
                  tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, is_figure = False, csv_end = "/" + title_save[i] + ".npy", normalize = normalize)
        path_best = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo, Bo_iter, title_save[i], run, tot_iter=tot_iter, 
                  tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, is_figure = False, csv_end = "/" + "GP_Best_EI_Pred.npy", normalize = normalize)
        path_opt = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo, Bo_iter, title_save[i], run, tot_iter=tot_iter, 
                  tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, is_figure = False, csv_end = "/" + "GP_Min_SSE_Pred.npy", normalize = normalize)
        path_true = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo, Bo_iter, title_save[i], run, tot_iter=tot_iter, 
                  tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, is_figure = False, csv_end = "/" + "True_p.npy", normalize = normalize)

        # print(p_GP_opt)
        z_i = np.load(path_title)

        if title == 'log(E(I(\\theta)))':
            z_i = np.exp(z_i)  
        z[i] = z_i

    # print(len(title), len(z))

    title_save_2 = "Train_Test_Data"
    #Need to fix this
    path_train = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, None, Bo_iter, title_save_2, run, 
                             tot_iter=tot_iter, tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, 
                             is_figure = False, csv_end = "/" + "Train_p.npy")
    path_test = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, None, Bo_iter, title_save_2, run, 
                             tot_iter=tot_iter, tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, 
                             is_figure = False, csv_end = "/" + "Test_p.npy")

    train_p = np.load(path_train)
    test_p = np.load(path_test)
    p_GP_best = np.load(path_best)
    p_GP_opt = np.load(path_opt)
    p_true = np.load(path_true)

    many_value_plotter(theta_mesh, z, p_true, p_GP_opt, p_GP_best, train_p, title, title_save, obj, ep, emulator, sparse_grid, 
                  set_lengthscale, save_figure, param_names_list, Bo_iter, run = run, tot_iter = tot_iter, tot_runs = tot_runs, 
                  DateTime=DateTime, t = t, sep_fact = sep_fact, levels = levels, save_CSV = save_CSV)

In [ ]:
levels = 20
save_figure = False
# save_figure = True
# titles = ['E(I(\\theta))','log(e(\\theta))','\sigma^2','\sigma','Best_Error','z','EI_term_1','EI_term_2','CDF','PDF']  
# titles_save = ["EI","ln(SSE)","Var","StDev","Best_Error","z","ei_term_1","ei_term_2","CDF","PDF"] 

fxn = "value_plotter"

title_save = ["EI", "ln(SSE)"]
title = ['E(I(\\theta))', 'log(e(\\theta))']

for i in range(len(title)):
    path_title = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo, Bo_iter, title_save[i], run, tot_iter=tot_iter, 
              tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, is_figure = False, csv_end = "/" + title_save[i] + ".npy", normalize = normalize)
    path_best = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo, Bo_iter, title_save[i], run, tot_iter=tot_iter, 
              tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, is_figure = False, csv_end = "/" + "GP_Best_EI_Pred.npy", normalize = normalize)
    path_opt = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo, Bo_iter, title_save[i], run, tot_iter=tot_iter, 
              tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, is_figure = False, csv_end = "/" + "GP_Min_SSE_Pred.npy", normalize = normalize)
    path_true = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo, Bo_iter, title_save[i], run, tot_iter=tot_iter, 
              tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, is_figure = False, csv_end = "/" + "True_p.npy", normalize = normalize)

    title_save_2 = "Train_Test_Data"
    #Need to fix this
    path_train = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, None, Bo_iter, title_save_2, run, 
                             tot_iter=tot_iter, tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, 
                             is_figure = False, csv_end = "/" + "Train_p.npy")
    path_test = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, None, Bo_iter, title_save_2, run, 
                             tot_iter=tot_iter, tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, 
                             is_figure = False, csv_end = "/" + "Test_p.npy")

    train_p = np.load(path_train)
    test_p = np.load(path_test)
    p_GP_best = np.load(path_best)
    p_GP_opt = np.load(path_opt)
    p_true = np.load(path_true)

    # print(p_GP_opt)
    z = np.load(path_title)

    if title == 'log(E(I(\\theta)))':
        z = np.exp(z)  

    value_plotter(theta_mesh, z, p_true, p_GP_opt, p_GP_best, train_p, title[i],title_save[i], obj, ep, emulator, sparse_grid, 
                  set_lengthscale, save_figure, param_names_list, Bo_iter, run = run, tot_iter = tot_iter, tot_runs = tot_runs, 
                  DateTime=DateTime, t = t, sep_fact = sep_fact, levels = levels, save_CSV = save_CSV)

In [ ]:
#WORKS
csv_end = ".npy" #For obj
fxn = "plot_obj"

path = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
          title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                 sep_fact = sep_fact, is_figure = is_figure, csv_end = ".npy", normalize = normalize)
# print(path)
obj_array = np.load(path)
# print(obj_array)
plot_obj(obj_array, t, obj, ep, emulator, sparse_grid, set_lengthscale, save_figure, tot_iter=tot_iter, 
         tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, save_CSV = save_CSV, normalize = normalize)

In [ ]:
#WORKS
csv_end = ".npy" #For obj
fxn = "plot_obj_abs_min"

path = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
          title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                 sep_fact = sep_fact, is_figure = is_figure, csv_end = ".npy", normalize = normalize)
# print(path)

obj_abs_min = np.load(path)

# obj_abs_min = obj_abs_min[:,0:8]
plot_obj_abs_min(obj_abs_min, emulator, ep, sparse_grid, set_lengthscale, t, obj,
                 save_figure, tot_iter, tot_runs,DateTime, sep_fact, save_CSV = save_CSV)

In [ ]:
#WORKS
# save_figure = True
Theta_array = []

fxn = "plot_Theta"
for i in range(len(param_dict.keys())):
    csv_end = "/"+param_dict[i]+ ".npy" #For Thetas #Need to ensure this saves correctly

    path_i = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
              title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                     sep_fact = sep_fact, is_figure = is_figure, csv_end = csv_end)
    Theta_array.append( np.load(path_i) )

Theta_array = np.array(Theta_array).T

plot_Theta(Theta_array, Theta_True, t, obj, ep, emulator, sparse_grid,  set_lengthscale, save_figure, 
           param_dict, tot_iter, tot_runs, DateTime, sep_fact = sep_fact)

In [ ]:
#WORKS
# save_figure = True
nbins = 10

Theta_array = []

fxn = "plot_Theta_min"
for i in range(len(param_dict.keys())):
    csv_end = "/"+param_dict[i]+ ".npy" #For Thetas #Need to ensure this saves correctly

    path_i = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
              title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                     sep_fact = sep_fact, is_figure = is_figure, csv_end = csv_end)
    Theta_array.append( np.load(path_i) )

Theta_array = np.array(Theta_array).T

plot_Theta_min(Theta_array, Theta_True, t, obj, ep, emulator, sparse_grid, set_lengthscale, save_figure, 
           param_dict, tot_iter=tot_iter, tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, nbins = nbins, save_CSV = save_CSV)

In [ ]:
fxn = "plot_org_train"
save_figure = False

path_train = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj,  mesh_combo, bo_iter=None, title_save = None, run = run, 
                       tot_iter=tot_iter, tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, is_figure = False, 
                       csv_end = "/" + "train_theta.npy")
path_test = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo, bo_iter=None, title_save = None, run = run, 
                       tot_iter=tot_iter, tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, is_figure = False, 
                       csv_end = "/" + "test_theta.npy")
fxn = "value_plotter"
path_true = path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo, Bo_iter, title_save, run, tot_iter=tot_iter, 
          tot_runs=tot_runs, DateTime=DateTime, sep_fact = sep_fact, is_figure = False, csv_end = "/" + "True_p.npy")

train_p = np.load(path_train)
test_p = np.load(path_test)
true_p = np.load(path_true)
plot_org_train(theta_mesh,train_p, test_p, true_p, Xexp, emulator, sparse_grid, obj, ep, set_lengthscale, run, save_figure, param_names_list, 
               tot_iter=tot_iter, tot_runs=tot_runs, DateTime=DateTime, verbose = True, sep_fact = sep_fact)